In [1]:
import h5py
import os
import subprocess

## Precursors

In [2]:
if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True)                

In [3]:
if not os.path.isdir('models/heart'):
    os.mkdir('models/heart')
if not os.path.isfile('models/heart/model_best.h5'):
    subprocess.call('curl -o models/heart/model_best.h5 https://storage.googleapis.com/basenji_tutorial_data/model_best.h5', shell=True)

In [4]:
lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'CNhs11760', 'data/CNhs11760.bw', '384', 'sum', 'aorta'])
lines.append(['1', 'CNhs12843', 'data/CNhs12843.bw', '384', 'sum', 'artery'])
lines.append(['2', 'CNhs12856', 'data/CNhs12856.bw', '384', 'sum', 'pulmonic_valve'])

samples_out = open('data/heart_wigs.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

## SNP activity difference compute

Analyzing noncoding variation associated with disease is a major application of Basenji. I now offer several tools to enable that analysis. If you have a small set of variants and know what datasets are most relevant, [basenji_sat_vcf.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_vcf.py) lets you perform a saturation mutagenesis of the variant and surrounding region to see the relevant nearby motifs.

If you want scores measuring the influence of those variants on all datasets,
 * [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) computes my SNP activity difference (SAD) score--the predicted change in aligned fragments to the region.
 * [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py) computes my SNP expression difference (SED) score--the predicted change in aligned fragments to gene TSS's.

Here, I'll demonstrate those two programs. You'll need
 * Trained model
 * Input file (FASTA or HDF5 with test_in/test_out)

First, you can either train your own model in the [Train/test tutorial](https://github.com/calico/basenji/blob/master/tutorials/train_test.ipynb) or use one that I pre-trained from the models subdirectory.

As an example, we'll study a prostate cancer susceptibility allele of rs339331 that increases RFX6 expression by modulating HOXB13 chromatin binding (http://www.nature.com/ng/journal/v46/n2/full/ng.2862.html).

First, we'll use [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) to predict across the region for each allele and compute stats about the mean and max differences.

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -f | data/hg19.ml.fa | Genome fasta. |
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| -o | rfx6_sad | Outplot plot directory. |
| --rc | True | Ensemble predictions for forward and reverse complement sequences. |
| --shift | 1,0,-1 | Ensemble predictions for sequences shifted by 1, 0, and -1 bp. |
| -t | data/heart_wigs.txt | Target labels. |
| params_file | models/params_small.json | JSON specified parameters to setup the model architecture and optimization. |
| model_file | models/heart/model_best.h5 | Trained saved model parameters. |
| vcf_file | data/rs339331.vcf | VCF file specifying variants to score. |

In [6]:
! basenji_sad.py -f data/hg19.ml.fa -o output/rfx6_sad --rc --shift "1,0,-1" -t data/heart_wigs.txt models/params_small.json models/heart/model_best.h5 data/rs339331.vcf

2021-02-12 15:49:10.346186: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-02-12 15:49:10.346327: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sequence (InputLayer)           [(None, 131072, 4)]  0                                            
__________________________________________________________________________________________________
stochastic_reverse_complement ( ((None, 131072, 4),  0           sequence[0][0]                   
__________

## SNP activity difference output

The output HDF5 stores the SNP and target information and predicted scores.

In [9]:
sad_h5 = h5py.File('output/rfx6_sad/sad.h5', 'r')
list(sad_h5.keys())

['SAD',
 'SAD_pct',
 'alt_allele',
 'chr',
 'percentiles',
 'pos',
 'ref_allele',
 'snp',
 'target_ids',
 'target_labels']

In [11]:
for snp_key in ['snp', 'chr', 'pos', 'ref_allele']:
    print(snp_key, sad_h5[snp_key][:])

snp [b'rs339331']
chr [b'chr6']
pos [117210052]
ref_allele [b'T']


In [12]:
for ti in range(3):
    cols = (ti, sad_h5['SAD'][0,ti], sad_h5['target_ids'][ti], sad_h5['target_labels'][ti])
    print('%2d  %7.4f  %12s  %s' % cols)

 0   0.0018  b'CNhs11760'  b'aorta'
 1  -0.0014  b'CNhs12843'  b'artery'
 2  -0.0028  b'CNhs12856'  b'pulmonic_valve'


These are inconclusive small effect sizes, not surprising given that we're only studying heart CAGE. The proper cell types and experiments would shed more light.

## SNP expression difference compute

Alternatively, we can directly query the predictions at gene TSS's. Since moving the codebase to Tensorflow2, these scripts need to be rewritten. In my experience thus far, the more general SNP scores above work well for nearly all applications. The model isn't terribly good at predicting that one gene versus another will be more affected right now.